# Model selection

### Read in and clean text

In [6]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

### Split into train/test

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data[['body_text', 'body_len', 'punct%']], data['label'], test_size=0.2)

### Vectorize text

In [8]:
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
tfidf_vect_fit = tfidf_vect.fit(X_train['body_text'])

tfidf_train = tfidf_vect_fit.transform(X_train['body_text'])
tfidf_test = tfidf_vect_fit.transform(X_test['body_text'])

In [9]:
X_train_vect = pd.concat([X_train[['body_len','punct%']].reset_index(drop=True), pd.DataFrame(tfidf_train.toarray())], axis=1)

In [10]:
X_test_vect = pd.concat([X_test[['body_len','punct%']].reset_index(drop=True), pd.DataFrame(tfidf_test.toarray())], axis=1)

In [13]:
print(X_train_vect.head())
print(X_test_vect.head())

   body_len  punct%         0    1    2    3    4    5    6    7  ...  7124  \
0       149     4.7  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   
1        53     5.7  0.238179  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   
2        24     8.3  0.217292  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   
3        39     5.1  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   
4        43     2.3  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   

   7125  7126  7127  7128  7129  7130  7131  7132  7133  
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
1   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
2   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
3   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
4   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  

[5 rows x 7136 columns]
   body_len  punct%         0    1    2    3    4    5    6    7  ...  7124  \
0        93     1.1  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0

### Final evaluation of models

In [14]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import precision_recall_fscore_support as score
import time

In [15]:
rf = RandomForestClassifier(n_estimators=150, max_depth=None, n_jobs=-1)

start = time.time()
rf_model = rf.fit(X_train_vect, y_train)
end= time.time()

fit_time = end - start

start = time.time()
y_pred = rf_model.predict(X_test_vect)
end = time.time()

pred_time = end - start

precision, recall, fscore, train_support = score(y_test, y_pred, pos_label='spam', average='binary')
print('Fit_time: {}, Pred_time:{} ,Precision: {} / Recall: {} / Accuracy: {}'.format(
    round(fit_time,3),round(pred_time,3),round(precision, 3), round(recall, 3), round((y_pred==y_test).sum()/len(y_pred), 3)))

Fit_time: 12.819, Pred_time:0.252 ,Precision: 1.0 / Recall: 0.861 / Accuracy: 0.98


In [16]:
gb = GradientBoostingClassifier(n_estimators=150, max_depth=11)

start = time.time()
gb_model = gb.fit(X_train_vect, y_train)
end= time.time()

fit_time = end - start

start = time.time()
y_pred = gb_model.predict(X_test_vect)
end = time.time()

pred_time = end - start

precision, recall, fscore, train_support = score(y_test, y_pred, pos_label='spam', average='binary')
print('Fit_time: {}, Pred_time:{} ,Precision: {} / Recall: {} / Accuracy: {}'.format(
    round(fit_time,3),round(pred_time,3),round(precision, 3), round(recall, 3), round((y_pred==y_test).sum()/len(y_pred), 3)))

Fit_time: 476.245, Pred_time:0.177 ,Precision: 0.932 / Recall: 0.873 / Accuracy: 0.973
